In [ ]:
!pip install python-dotenv

In [ ]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col, lit,  date_format, to_date, hour, minute
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql import functions as F
from dotenv import load_dotenv
import os

# Load variables from .env file
load_dotenv()

# Access sensitive variables
es_url = os.getenv("ELASTICSEARCH")
es_username = os.getenv("ELASTICSEARCH_USERNAME")
es_password = os.getenv("ELASTICSEARCH_PASSWORD")

spark = SparkSession.builder.appName("REST_API_with_PySpark_DF").getOrCreate()

# Important parameters can be found here
# Some have to be manually modified within the code
schema = StructType([
    StructField("@timestamp", StringType(), True),
    StructField("source.address", StringType(), True),
])
days_to_fetch = 7  # Specify the number of days to fetch

@udf(returnType=ArrayType(schema))
def fetch_data(offset: int, limit: int, days: int):
    endpoint = "https://"+es_url+"/filebeat-*/_search"

    # Calculate start date based on the specified number of days
    import datetime
    start_date = (datetime.datetime.now() - datetime.timedelta(days=days)).isoformat()

    # Get field names from the schema
    fields = [field.name for field in schema.fields]

    # Elasticsearch query with time filter, proper pagination, and selected fields
    es_query = {
        "size": limit,
        "from": offset,
        "_source": fields,  # Select only specified fields
        "query": {
            "bool": {
                "must": [
                    {
                        "term": {
                            "suricata.eve.event_type": "dns"
                        }
                    },
                    {
                        "term": {
                            "network.direction": "outbound"
                        }
                    },
                    {
                        "range": {
                            "@timestamp": {
                                "gte": start_date
                            }
                        }
                    }
                ]
            }
        }
    }

    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.get(endpoint, json=es_query, headers=headers, verify=False, auth=(es_username, es_password))

    # Extract hits from the response
    hits = response.json().get('hits', {}).get('hits', [])

    # Extract necessary fields from hits and create a list of records
    records = [{"@timestamp": hit.get('_source', {}).get('@timestamp'),
                "source.address": hit.get('_source', {}).get('source', {}).get('address')}
               for hit in hits]

    return records  # assuming API returns a list of records

# Get total docs
total_records = requests.get("https://"+es_url+"/filebeat-*/_count", verify=False, auth=(es_username, es_password)).json().get('count', 0)

records_per_page = 500

# Create DataFrame with pagination information
offsets_df = spark.range(0, total_records, records_per_page).select(col("id").alias("offset"), lit(records_per_page).alias("limit"))

# Apply fetch_data UDF to get the response with time filter
response_df = offsets_df.withColumn("response", explode(fetch_data("offset", "limit", lit(days_to_fetch))))

# Uncomment the lines above if you want to further explode and select individual fields
# response_df.show(truncate=False)

# Extract variables using positional indexing
result_df = response_df.select(
#    "offset",
#    "limit",
    col("response")["@timestamp"].alias("@timestamp"),
    col("response")["source.address"].alias("source.address")
)

result_df.count()

In [ ]:
# Feature Engineering
features_df = result_df.withColumn("day", F.dayofmonth(F.col("@timestamp")))

# Calculate the fraction of the day that has passed
current_time = F.current_timestamp()
fraction_of_day = (F.hour(current_time) * 60 + F.minute(current_time)) / (24 * 60)

# Add the fraction_of_day as a new feature
features_df = features_df.withColumn("fraction_of_day", lit(fraction_of_day))

# Aggregating features by day and source address
aggregated_df = features_df.groupBy("day", "`source.address`").agg(
    F.count("*").alias("request_count"),
    F.avg("fraction_of_day").alias("avg_fraction_of_day")
)

# Filter data for the past 6 days
current_day = features_df.select(F.max("day")).first()[0]
training_data = aggregated_df.filter((col("day") >= current_day - 6) & (col("day") < current_day + 1))

# Machine Learning
assembler = VectorAssembler(inputCols=["day", "request_count", "avg_fraction_of_day"], outputCol="features")
regressor = RandomForestRegressor(featuresCol="features", labelCol="request_count", predictionCol="prediction")

# Creating a Pipeline
pipeline = Pipeline(stages=[assembler, regressor])

# Training the model on the past 6 days
model = pipeline.fit(training_data)

# Predicting on the current day
current_day_data = aggregated_df.filter(F.col("day") == current_day).withColumn("fraction_of_day", lit(fraction_of_day))
prediction_df = model.transform(current_day_data)

# Displaying the actual and predicted values
prediction_df.select("`source.address`", "request_count", "prediction").show(truncate=False)


In [ ]:
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt

# Extracting actual and predicted values
source_ips = prediction_df.select("`source.address`").rdd.flatMap(lambda x: x).collect()
actual_values = prediction_df.select("request_count").rdd.flatMap(lambda x: x).collect()
predicted_values = prediction_df.select("prediction").rdd.flatMap(lambda x: x).collect()

# Perform t-test for paired samples
t_statistic, p_value = ttest_rel(actual_values, predicted_values)

# Display the t-test results
print(f"T-Statistic: {t_statistic}")
print(f"P-Value: {p_value}")

# Visualization - Bar plot per source IP
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.35
index = range(len(source_ips))

bar1 = ax.bar(index, actual_values, bar_width, label='Actual', color='blue')
bar2 = ax.bar(index, predicted_values, bar_width, label='Predicted', color='orange', alpha=0.7)

ax.set_xlabel('Source IP')
ax.set_ylabel('Count')
ax.set_title('Actual vs. Predicted Request Count per Source IP')
ax.set_xticks(index)
ax.set_xticklabels(source_ips, rotation=45)
ax.legend()

# Check if the difference is statistically significant and raise an alert
alpha = 0.05
for i in range(len(source_ips)):
    if actual_values[i] > predicted_values[i] and p_value < alpha:
        print(f"Alert: The difference for Source IP {source_ips[i]} is statistically significant. Actual is above predicted.")
    else:
        print(f"No alert for Source IP {source_ips[i]}")

plt.show()
